In [24]:
from __future__ import print_function
import logging
import torch
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
import os
import platform
import time
import sys
sys.path.append('..')
from utils.Network import Network
from utils.Analyser import Analyser
from utils.io import save_network, load_network, save, load, make_folder_results
from utils.WaveDataset import create_datasets, transformVar
from utils.training import train_epoch, validate, test
from utils.arg_parse import get_args

In [25]:
# normalize = transforms.Normalize(mean=[0.5047], std=[0.1176])

transformVar = {"Test": transforms.Compose([
    transforms.Resize(128),    #Already 184 x 184
    transforms.CenterCrop(128),
    transforms.ToTensor(),
#     normalize
]),
    "Train": transforms.Compose([
    transforms.Resize(128),  # Already 184 x 184
    transforms.CenterCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
#     normalize
    ])
}

In [12]:
data_dir = '../'
num_channels=1
num_workers=12

test_dataset, val_dataset, train_dataset = create_datasets(
     data_dir+"Video_Data/", transformVar, test_fraction=0.01, validation_fraction=0.01, check_bad_data=False, channels=num_channels)
all_data = {"Training data": train_dataset, "Validation data": val_dataset, "Testing data": test_dataset}
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=num_workers)

In [13]:
bb = next(iter(train_dataloader))

In [14]:
bb['image'].max()

tensor(1.)

In [16]:
mean=0.
std=0.
nb_samples=0

for bb in train_dataloader:
    data=bb['image']
    batch_samples = data.size(0)
    images_per_video = data.size(1)
    data = data.view(batch_samples, images_per_video, -1)
    mean += data.mean(2).sum()
    std += data.std(2).sum()

    nb_samples += batch_samples*images_per_video
 
mean /= nb_samples
std /= nb_samples

In [17]:
mean

tensor(0.5047)

In [18]:
std

tensor(0.1176)

In [19]:
nb_samples

320900

In [20]:
16*100*len(train_dataloader)

321600

In [21]:
num_channels=3
num_workers=12

test_dataset, val_dataset, train_dataset = create_datasets(
     data_dir+"Video_Data/", transformVar, test_fraction=0.01, validation_fraction=0.01, check_bad_data=False, channels=num_channels)
all_data = {"Training data": train_dataset, "Validation data": val_dataset, "Testing data": test_dataset}
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=num_workers)

In [29]:
mean=0.
std=0.
nb_samples=0

for bb in train_dataloader:
    data=bb['image']
    batch_samples = data.size(0)
    images_per_video = data.size(1)
    data = data.view(batch_samples, int(images_per_video/3), 3, -1)
    mean += data.mean(3).sum(0).sum(0)
    std += data.std(3).sum(0).sum(0)

    nb_samples += batch_samples*images_per_video
 
mean /= nb_samples
std /= nb_samples

In [30]:
mean

tensor([0.1683, 0.1683, 0.1683])

In [31]:
std

tensor([0.0392, 0.0392, 0.0392])

In [32]:
nb_samples

962700

In [33]:
bb = next(iter(train_dataloader))

In [34]:
data=bb['image']

In [35]:
data.view(16,100,3,-1).mean(3).sum(0).sum(0)

tensor([810.1906, 810.1906, 810.1906])